In [1]:
from typing import Dict, Tuple

import eurostat  # docs: https://bitbucket.org/noemicazzaniga/eurostat/src/master/
import pandas as pd
import torch
from InstructorEmbedding import INSTRUCTOR # https://instructor-embedding.github.io/
from pandas import DataFrame
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer

/work/davor/projects/airt/eurostat-embedding/.env/lib/python3.11/site-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


In [2]:
toc = eurostat.get_toc_df()
toc.head().style

,title,code,type,last update of data,last table structure change,data start,data end
0,Passengers (excluding cruise passengers) transported from/to the main ports - Netherlands - quarterly data,MAR_PA_QM_NL,dataset,2010-02-26T23:00:00+0100,2023-07-28T23:00:00+0200,1997-Q1,2022-Q4
1,Passengers (excluding cruise passengers) transported from/to the main ports - Norway - quarterly data,MAR_PA_QM_NO,dataset,2023-10-17T23:00:00+0200,2023-10-17T23:00:00+0200,2002-Q1,2022-Q4
2,Passengers (excluding cruise passengers) transported from/to the main ports - Poland - quarterly data,MAR_PA_QM_PL,dataset,2023-09-12T11:00:00+0200,2023-09-12T11:00:00+0200,2004-Q1,2023-Q1
3,Passengers (excluding cruise passengers) transported from/to the main ports - Portugal - quarterly data,MAR_PA_QM_PT,dataset,2023-09-19T11:00:00+0200,2023-09-19T11:00:00+0200,2000-Q1,2023-Q1
4,Passengers (excluding cruise passengers) transported from/to the main ports - Sweden - quarterly data,MAR_PA_QM_SE,dataset,2010-02-26T23:00:00+0100,2023-07-28T23:00:00+0200,1997-Q1,2022-Q4


In [3]:
toc

,title,code,type,last update of data,last table structure change,data start,data end
0,Passengers (excluding cruise passengers) trans...,MAR_PA_QM_NL,dataset,2010-02-26T23:00:00+0100,2023-07-28T23:00:00+0200,1997-Q1,2022-Q4
1,Passengers (excluding cruise passengers) trans...,MAR_PA_QM_NO,dataset,2023-10-17T23:00:00+0200,2023-10-17T23:00:00+0200,2002-Q1,2022-Q4
2,Passengers (excluding cruise passengers) trans...,MAR_PA_QM_PL,dataset,2023-09-12T11:00:00+0200,2023-09-12T11:00:00+0200,2004-Q1,2023-Q1
3,Passengers (excluding cruise passengers) trans...,MAR_PA_QM_PT,dataset,2023-09-19T11:00:00+0200,2023-09-19T11:00:00+0200,2000-Q1,2023-Q1
4,Passengers (excluding cruise passengers) trans...,MAR_PA_QM_SE,dataset,2010-02-26T23:00:00+0100,2023-07-28T23:00:00+0200,1997-Q1,2022-Q4
...,...,...,...,...,...,...,...
7549,Percentage of letters delivered on-time (USP u...,POST_CUBE1_X$POST_QOS_1,dataset,2023-05-23T08:18:37Z,2023-05-10T10:54:33Z,None,None
7550,Postal services,POST_CUBE1_X,dataset,2023-05-23T08:18:37Z,2023-05-10T10:54:33Z,2012,2021
7551,Number of enterprises providing postal services,POST_CUBE1_X$NUM701,dataset,2023-05-23T08:18:37Z,2023-05-10T10:54:33Z,None,None
7552,Access points (USP under direct or indirect d...,POST_CUBE1_X$POST_ACC_1,dataset,2023-05-23T08:18:37Z,2023-05-10T10:54:33Z,None,None


In [4]:
code = "demo_r_d2jan"
data = eurostat.get_data_df(code)
data

,freq,unit,sex,age,geo\TIME_PERIOD,1990,1991,1992,1993,1994,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,A,NR,F,TOTAL,AL,NaN,NaN,NaN,NaN,NaN,...,1437193.0,1430827.0,1424597.0,1417141.0,1423050.0,1431715.0,1432833.0,1425342.0,1419759.0,1406532.0
1,A,NR,F,TOTAL,AL0,NaN,NaN,NaN,NaN,NaN,...,1437193.0,1430827.0,1424597.0,1417141.0,1423050.0,1431715.0,1432833.0,1425342.0,1419759.0,1406532.0
2,A,NR,F,TOTAL,AL01,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,416845.0,405835.0,405598.0,404201.0,399599.0,396799.0,390886.0
3,A,NR,F,TOTAL,AL02,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,552949.0,574010.0,585530.0,590623.0,594008.0,596005.0,597622.0
4,A,NR,F,TOTAL,AL03,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,454906.0,443205.0,440587.0,438009.0,431735.0,426955.0,418024.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157528,A,NR,T,Y_OPEN,UKM7,NaN,NaN,NaN,NaN,NaN,...,294.0,315.0,333.0,342.0,343.0,323.0,303.0,NaN,NaN,NaN
157529,A,NR,T,Y_OPEN,UKM8,NaN,NaN,NaN,NaN,NaN,...,206.0,221.0,229.0,231.0,229.0,210.0,193.0,NaN,NaN,NaN
157530,A,NR,T,Y_OPEN,UKM9,NaN,NaN,NaN,NaN,NaN,...,149.0,161.0,166.0,164.0,159.0,160.0,151.0,NaN,NaN,NaN
157531,A,NR,T,Y_OPEN,UKN,NaN,NaN,17566.0,17934.0,18214.0,...,237.0,250.0,265.0,274.0,281.0,282.0,NaN,NaN,NaN,NaN


In [5]:
pars = eurostat.get_pars(code)
pars

['freq', 'unit', 'sex', 'age', 'geo']

In [6]:
for par in pars:
    print(par)
    d = eurostat.get_dic(code, par, frmt="dict")
    df = pd.DataFrame.from_dict(d, orient="index")
    display(df)
# d = eurostat.get_dic(code, par, frmt="dict")
# d

freq


,0
P,Pluri-annual
A,Annual
S,"Half-yearly, semesterly"
Q,Quarterly
M,Monthly
W,Weekly
B,Daily - business week
D,Daily
H,Hourly
I,Irregular / A-periodic


unit


,0
TOTAL,Total
NR,Number
NR_HAB,Number per inhabitant
THS,Thousand
MIO,Million
...,...
PD_PCH_SM_NAC,"Price index (implicit deflator), percentage ch..."
CRC_MEUR,"Current replacement costs, million euro"
CRC_MNAC,"Current replacement costs, million units of na..."
PYR_MEUR,"Previous year replacement costs, million euro"


sex


,0
T,Total
M,Males
F,Females
DIFF,Absolute difference between males and females
NAP,Not applicable
NRP,No response
UNK,Unknown


age


,0
TOTAL,Total
LFD,Late foetal death
LFD1,Late foetal death (group 1)
LFD2,Late foetal death (group 2)
MN0,Zero minutes
...,...
AVG,Average
NRP,No response
NSP,Not specified
OTH,Other


geo


,0
EUR,Europe
EU,"European Union (EU6-1958, EU9-1973, EU10-1981,..."
EU_V,European Union (aggregate changing according t...
EU27_2020_EFTA,European Union - 27 countries (from 2020) and ...
EU27_2020_IS_K,European Union - 27 countries (from 2020) and ...
...,...
NAL,Not allocated
NAP,Not applicable
NRP,No response
NSP,Not specified


In [7]:
eurostat.get_data_df(code, flags=False, verbose=True, reverse_time=False)

Download progress: 100.0%



,freq,unit,sex,age,geo\TIME_PERIOD,1990,1991,1992,1993,1994,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,A,NR,F,TOTAL,AL,NaN,NaN,NaN,NaN,NaN,...,1437193.0,1430827.0,1424597.0,1417141.0,1423050.0,1431715.0,1432833.0,1425342.0,1419759.0,1406532.0
1,A,NR,F,TOTAL,AL0,NaN,NaN,NaN,NaN,NaN,...,1437193.0,1430827.0,1424597.0,1417141.0,1423050.0,1431715.0,1432833.0,1425342.0,1419759.0,1406532.0
2,A,NR,F,TOTAL,AL01,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,416845.0,405835.0,405598.0,404201.0,399599.0,396799.0,390886.0
3,A,NR,F,TOTAL,AL02,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,552949.0,574010.0,585530.0,590623.0,594008.0,596005.0,597622.0
4,A,NR,F,TOTAL,AL03,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,454906.0,443205.0,440587.0,438009.0,431735.0,426955.0,418024.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157528,A,NR,T,Y_OPEN,UKM7,NaN,NaN,NaN,NaN,NaN,...,294.0,315.0,333.0,342.0,343.0,323.0,303.0,NaN,NaN,NaN
157529,A,NR,T,Y_OPEN,UKM8,NaN,NaN,NaN,NaN,NaN,...,206.0,221.0,229.0,231.0,229.0,210.0,193.0,NaN,NaN,NaN
157530,A,NR,T,Y_OPEN,UKM9,NaN,NaN,NaN,NaN,NaN,...,149.0,161.0,166.0,164.0,159.0,160.0,151.0,NaN,NaN,NaN
157531,A,NR,T,Y_OPEN,UKN,NaN,NaN,17566.0,17934.0,18214.0,...,237.0,250.0,265.0,274.0,281.0,282.0,NaN,NaN,NaN,NaN


In [8]:
def get_dataset(code: str, toc: pd.DataFrame = toc) -> pd.DataFrame:
    """Get a Eurostat dataset as a Pandas DataFrame.

    Args:
        code (str): Eurostat dataset code.

    Returns:
        pd.DataFrame: Eurostat dataset as a Pandas DataFrame.
    """
    title = toc[toc["code"] == code.upper()]["title"].values[0]
    # print(f"{title=}")
    df = eurostat.get_data_df(code)
    df = df.rename(columns={r"geo\TIME_PERIOD": "geo"})
    # pars = eurostat.get_pars(code)
    print(f"{pars=}")
    for par in pars:
        right = pd.Series(eurostat.get_dic(code, par, frmt="dict"), name=par)
        # display(df)
        # display(right)
        df = df.merge(right, left_on=par, right_index=True, suffixes=("_x", None)).drop(
            columns=f"{par}_x"
        )
        # df = df.sort_index()
        # break
        # display(right)

    df = df.sort_values(by=pars)
    df = df.set_index(pars)

    df = df.stack()
    df.index.names = df.index.names[:-1] + ["time"]
    df.name = title

    mean, std = df.describe().loc[["mean", "std"]].values
    df = (df - mean) / std

    return df.to_frame()

In [9]:
df = get_dataset("demo_r_d2jan")
df.sample(25)

pars=['freq', 'unit', 'sex', 'age', 'geo']


Population on 1 January by age, sex and NUTS 2 region
freq   unit   sex     age      geo                                                time                                                       
Annual Number Total   17 years Sweden                                             2002                                           0.005396    
              Females 59 years Pest                                               2016                                          -0.028162    
              Total   25 years Corse                                              1995                                          -0.030006    
                      9 years  Champagne-Ardenne                                  2001                                          -0.024967    
                      51 years Nordrhein-Westfalen                                2020                                           0.073789    
              Females 1 year   Åland                                              2005                                          -0.031110    
              Total   61 years Stredné Slovensko                                  1998                                          -0.027337    
              Males   59 years Kentriki Ellada                                    2019                                          -0.024731    
              Females 83 years Sur (ES)                                           2011                                          -0.022495    
              Males   52 years Eesti                                              1996                                          -0.028905    
              Total   45 years Grand Est                                          2019                                          -0.004795    
              Females 41 years Noroeste (ES)                                      1990                                          -0.020559    
              Total   33 years Bursa, Eskisehir, Bilecik                          2021                                          -0.007815    
                      2 years  Lincolnshire                                       2018                                          -0.028252    
              Females 92 years Malopolskie                                        2017                                          -0.030335    
              Males   45 years Austria                                            2019                                          -0.010312    
                      31 years Outer London - East and North East                 2009                                          -0.026552    
                      2 years  Jadranska Hrvatska                                 2015                                          -0.028652    
              Females 69 years Sud-Est                                            2022                                          -0.024850    
              Total   31 years Région de Bruxelles-Capitale/Brussels Hoofdsted... 2019                                          -0.023611    
              Females 11 years Región de Murcia                                   2002                                          -0.028620    
              Total   76 years Zachodniopomorskie                                 1993                                          -0.029780    
                      7 years  Galicia                                            2018                                          -0.023104    
              Males   97 years Croatia                                            2009                                          -0.031136    
              Total   31 years Nord-Vest                                          2021                                          -0.016549

In [10]:
print(f"Table size: {df.shape[0]:,d}")

Table size: 4,254,758


In [11]:
# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [12]:
def tokenize(
    df: DataFrame, tokenizer: AutoTokenizer
) -> Tuple[Dict[str, torch.Tensor], torch.Tensor]:
    """Tokenize a DataFrame.

    Args:
        df (DataFrame): DataFrame to tokenize.
        tokenizer (AutoTokenizer): Tokenizer to use.

    Returns:
        DataFrame: Tokenized DataFrame.
    """
    feature_names = df.index.names
    tokens = {c: df.reset_index()[c].unique() for c in feature_names}
    tokens = {
        c: {
            x: tokenizer(x, padding=True, truncation=True, return_tensors="pt")
            for x in tokens[c]
        }
        for c in tokens.keys()
    }

    features = {
        c: pd.merge(
            left=df.reset_index()[c],
            right=pd.Series(tokens[c], name=c),
            how="right",
            left_on=c,
            right_index=True,
            suffixes=["_x", None],
        )
        .drop(columns=f"{c}_x")
        .values
        for c in feature_names
    }
    targets = torch.tensor(df.values)

    return features, targets

In [13]:
features, targets = tokenize(df, tokenizer)
features

{'freq': array([[{'input_ids': tensor([[ 101, 3296,  102]]), 'token_type_ids': tensor([[0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1]])}],
        [{'input_ids': tensor([[ 101, 3296,  102]]), 'token_type_ids': tensor([[0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1]])}],
        [{'input_ids': tensor([[ 101, 3296,  102]]), 'token_type_ids': tensor([[0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1]])}],
        ...,
        [{'input_ids': tensor([[ 101, 3296,  102]]), 'token_type_ids': tensor([[0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1]])}],
        [{'input_ids': tensor([[ 101, 3296,  102]]), 'token_type_ids': tensor([[0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1]])}],
        [{'input_ids': tensor([[ 101, 3296,  102]]), 'token_type_ids': tensor([[0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1]])}]],
       dtype=object),
 'unit': array([[{'input_ids': tensor([[ 101, 2193,  102]]), 'token_type_ids': tensor([[0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1]])}],
        [{'input_

In [15]:
class CustomDataset(Dataset):
    def __init__(self, df: DataFrame, tokenizer: AutoTokenizer):
        self.features, self.target = tokenize(df, tokenizer)

    def __len__(self):
        return len(self.target)

    def __getitem__(self, idx):
        return {key: tensor[idx] for key, tensor in self.features.items()}, self.target[
            idx
        ]

In [ ]:
# Create the Dataset
dataset = CustomDataset(df, tokenizer)

# Create the DataLoader
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

In [ ]:
# Iterate over the DataLoader
for i, batch in enumerate(dataloader):
    print(f"Batch {i}:")
    print(batch)

    # For demonstration purposes, break after the first batch
    if i == 0:
        break

In [16]:
model = INSTRUCTOR("hkunlp/instructor-large")

load INSTRUCTOR_Transformer
max_seq_length  512


/work/davor/projects/airt/eurostat-embedding/.env/lib/python3.11/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
